In [1]:
import numpy as np 
import pandas as pd 

import os
import re
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/dogus-teknoloji-n11/train_n11.csv
/kaggle/input/dogus-teknoloji-n11/test_n11.csv
/kaggle/input/dogus-teknoloji-n11/sample_submission_n11.csv
/kaggle/input/zemberekwords/stop-words.tr.txt


In [2]:
sample_submission = pd.read_csv('../input/dogus-teknoloji-n11/sample_submission_n11.csv')
test_data = pd.read_csv('../input/dogus-teknoloji-n11/test_n11.csv',sep='|')
train_data = pd.read_csv('../input/dogus-teknoloji-n11/train_n11.csv',sep='|')

In [3]:
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from nltk.tokenize import word_tokenize
import fasttext

In [4]:
with open('../input/zemberekwords/stop-words.tr.txt', 'r') as f:
    stop_words = [line.strip() for line in f]
    missing = ('sayede','mükemmel',)
    stop_words.extend(missing)    

In [5]:
def toClean(text):

    text = re.sub(r"I", "ı", text)
    text = text.lower()
    text = re.sub('ürün detayları', '', text)
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r"\n", "  ", text)
    text = re.sub(r"  ", " ", text)
    text = re.sub(' +', ' ', text)
    text = text.strip()
    text = re.sub(r'[^\w\s]',' ',text)
    return text

def cleanStopWords(text):
    words = word_tokenize(text)
    filtered_sentence = " ".join(w for w in words if not w in stop_words)
    return filtered_sentence


In [6]:
train_data['clean_des'] = train_data['DESCRIPTION'].apply(toClean)
train_data['clean_title'] = train_data['TITLE'].apply(toClean)
train_data['clean_des'] = train_data['clean_des'].apply(cleanStopWords)
train_data['clean_title'] = train_data['clean_title'].apply(cleanStopWords)
train_data['sumOfBag'] = train_data['clean_des'] + ' . ' + train_data['clean_title'] 
df = train_data['sumOfBag']

#train_data.CATEGORY_ID.value_counts().to_frame().head()

/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:343: MarkupResemblesLocatorWarning: "." looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


In [7]:
count_vec = TfidfVectorizer(max_features=5000)

bow = count_vec.fit_transform(df)
bow = np.array(bow.todense())

y = train_data['CATEGORY_ID']

In [8]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(bow,y)

SVC(gamma='auto', kernel='linear')

In [9]:
test_data['clean_des'] = test_data['DESCRIPTION'].apply(toClean)
test_data['clean_title'] = test_data['TITLE'].apply(toClean)
test_data['clean_des'] = test_data['clean_des'].apply(cleanStopWords)
test_data['clean_title'] = test_data['clean_title'].apply(cleanStopWords)
test_data['sumOfBag'] = test_data['clean_des'] + ' . ' + test_data['clean_title'] 
df2 = test_data['sumOfBag']

In [10]:
bow2 = count_vec.transform(df2)
bow2 = np.array(bow2.todense())

expected = SVM.predict(bow2)

In [11]:
test_id = test_data['ID']
expected_df = pd.DataFrame(expected)
result = pd.concat([test_id, expected_df], axis=1)
result.columns = ['ID','Expected']
result.to_csv(r'submission.csv', index = False)